<strong>Analise de Sentimentos Fazendo Join de duas bases pré-classificadas em português</strong>
<br>
<br>
(1) Resenhas: https://www.linguateca.pt/Repositorio/ReLi/
<br>
<br>
(2) Tweets: https://www.kaggle.com/leandrodoze/sentiment-analysis-in-portuguese
<br>
<br>
<strong> Inspirador Por:</strong> (2)

In [1]:
#Importando as bibliotecas necessárias a essa atividade
import os
import re
import pandas as pd
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [2]:
#Leitura dos arquivos da coleção ReLi
base_path_reli = 'ReLi-Lex'
train = []
files = [os.path.join(base_path_reli, f) for f in os.listdir(base_path_reli)]

for file in files:
    t = 'pos' if '_Positivos' in file else 'neg'
    with open(file, 'r') as content_file:
        content = content_file.read()
        all = re.findall('\[.*?\]',content)
        for w in all:
            train.append((w[1:-1], t))

In [3]:
#Transformação da coleção ReLi em dicionário, substituindo o rótulo neg para Negativo e pos para Positivo
#Dessa forma, os rótulos serão igualados a outra coleção
train_dict = {}
for x, y in train:
    if y == "neg":
        train_dict[x] = "Negativo"
    elif y == "pos":
        train_dict[x] = "Positivo"

In [4]:
#Para efeitos de estudo, é sempre bom saber com o que estamos lidando
print(type(train_dict), len(train_dict))

<class 'dict'> 597


In [5]:
#Transformando a coleção do ReLi em dataFrame
df_resenhas = pd.DataFrame(data=[[item[0], item[1]] for item in train_dict.items()], columns=['Text', 'Classificacao'])
df_resenhas.head()

,Text,Classificacao
0,aborrecente,Negativo
1,anacrônico,Negativo
2,besta,Negativo
3,bizarro,Negativo
4,bobo,Negativo


In [6]:
#Leitura dos Tweets rotulados para análise de sentimentos
df_tweets = pd.read_csv('Tweets_Mg.csv',encoding='utf-8', usecols=["Text", "Classificacao"])
df_tweets.count()

Text             8199
Classificacao    8199
dtype: int64

In [7]:
#Verificando como estão os nossos dados
df_tweets.head()

,Text,Classificacao
0,���⛪ @ Catedral de Santo Antônio - Governador ...,Neutro
1,"� @ Governador Valadares, Minas Gerais https:/...",Neutro
2,"�� @ Governador Valadares, Minas Gerais https:...",Neutro
3,��� https://t.co/BnDsO34qK0,Neutro
4,��� PSOL vai questionar aumento de vereadores ...,Negativo


In [8]:
#Realizando a contagem de cada classe, em outros trabalhos talvez seja interessante ter classes balanceadas.
qtd_neutros   = df_tweets[df_tweets.Classificacao == "Neutro"]
qtd_positivos = df_tweets[df_tweets.Classificacao == 'Positivo']
qtd_negativos = df_tweets[df_tweets.Classificacao == 'Negativo']
print(10*"=","Tweets Neutros",10*"=")
print(qtd_neutros.count())
print()

print(10*"=","Tweets Positivos",10*"=")
print(qtd_positivos.count())
print()

print(10*"=","Tweets Negativos",10*"=")
print(qtd_negativos.count())

========== Tweets Neutros ==========
Text             2453
Classificacao    2453
dtype: int64

========== Tweets Positivos ==========
Text             3300
Classificacao    3300
dtype: int64

========== Tweets Negativos ==========
Text             2446
Classificacao    2446
dtype: int64


In [9]:
#Concatenando os dois DataSets
frames = [df_resenhas, df_tweets]
train_set = pd.concat(frames)

In [10]:
#Descrever a coleção ajuda a compreender possíveis melhorias
train_set.describe()

,Text,Classificacao
count,8796,8796
unique,6362,3
top,RT @AnaPaulaVolei: Mais 2 helicópteros!!A cara...,Positivo
freq,300,3679


In [11]:
#Como observado a duplicade de de registros, foi necessário a remoção de registros duplicados.
train_set.drop_duplicates(subset='Text', inplace=True)
train_set.describe()

,Text,Classificacao
count,6362,6362
unique,6362,3
top,"Banco do Brasil cobra de Minas R$ 1,5 bilhão e...",Positivo
freq,1,3219


In [12]:
#Por desencargo de conciência, realizei a remoção de registros nulos, mas não resultou em nada
train_set.dropna(inplace=True)
train_set.describe()

,Text,Classificacao
count,6362,6362
unique,6362,3
top,"Banco do Brasil cobra de Minas R$ 1,5 bilhão e...",Positivo
freq,1,3219


In [13]:
#Recontando os registros por classe, agora com a coleção concatenada e higienizada
qtd_neutros   = train_set[train_set.Classificacao == "Neutro"]
qtd_positivos = train_set[train_set.Classificacao == 'Positivo']
qtd_negativos = train_set[train_set.Classificacao == 'Negativo']
print(10*"=","Tweets Neutros",10*"=")
print(qtd_neutros.count())
print()

print(10*"=","Tweets Positivos",10*"=")
print(qtd_positivos.count())
print()

print(10*"=","Tweets Negativos",10*"=")
print(qtd_negativos.count())

========== Tweets Neutros ==========
Text             1974
Classificacao    1974
dtype: int64

========== Tweets Positivos ==========
Text             3219
Classificacao    3219
dtype: int64

========== Tweets Negativos ==========
Text             1169
Classificacao    1169
dtype: int64


In [14]:
# Remoção de Acentos e Caracteres Especiais - Padronização de Texto\n
dicionario_traducao = {'#': '', '{': '', '}': '', '>': '', '<': '', ',': '', ';': '',
                       '\"': '','@': '','&': '','®': '','¿': '','!': '','/': '','-':''}
train_set.replace(dicionario_traducao, regex=True, inplace=True)
train_set.describe()

,Text,Classificacao
count,6362,6362
unique,6361,3
top,E governo ainda quer indenizar a família dos b...,Positivo
freq,2,3219


# Naive Bayes

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

#Criação do modelo
from sklearn.pipeline import make_pipeline
modelo = make_pipeline(TfidfVectorizer(analyzer = "word"),
                       MultinomialNB())

classifier = GridSearchCV(estimator=modelo, 
                          param_grid={
                              'multinomialnb__alpha':[0.1, 0.5, 1.0], 
                              'multinomialnb__fit_prior':[True,False], 
                              'tfidfvectorizer__ngram_range':[(1,1), (1,2), (2,2)]
                          },                           
                          n_jobs=4,
                          cv=10)

X = train_set['Text']
y = train_set['Classificacao']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

classifier.fit(X_train, y_train)

predicoes = classifier.predict(X_test)

np.mean(predicoes == y_test)

0.9057142857142857

In [16]:
from sklearn import metrics
print(metrics.classification_report(y_test, predicoes))
print()
print("Melhores Parâmetros:", classifier.best_params_)

              precision    recall  f1-score   support

    Negativo       0.89      0.76      0.82       411
      Neutro       0.89      0.91      0.90       628
    Positivo       0.92      0.96      0.94      1061

    accuracy                           0.91      2100
   macro avg       0.90      0.88      0.89      2100
weighted avg       0.91      0.91      0.90      2100


Melhores Parâmetros: {'multinomialnb__alpha': 0.1, 'multinomialnb__fit_prior': True, 'tfidfvectorizer__ngram_range': (1, 2)}


In [17]:
#Persistindo o modelo para publicação futura em uma API
from joblib import dump, load
dump(classifier, 'SentimentAnalysisNB.pkl')

['SentimentAnalysisNB.pkl']

In [18]:
#Reabrindo o modelo e validando em algumas frases
modelo = load('SentimentAnalysisNB.pkl')

validacao = ["Existem muitas formas de se aprender, basta querer.",
          "Há dias que o trabalho é muito cansativo.", 
          "Hoje não estou muito bem",
          "Depois do culto você vai ver",
          "Vestidos de heróis dos pés a cabeça para levar um pouco de conhecimento para a humanidade",
          "A Net é muito boa, tem um excelente serviço de telefonia, sucesso de vendas"
         ]
resultado = modelo.predict(validacao)
predicted_proba = modelo.predict_proba(validacao)
for i in range(len(validacao)):
    print(str(i),"\t",resultado[i],"\t",predicted_proba[i])

0 	 Negativo 	 [0.46097559 0.42890592 0.11011849]
1 	 Neutro 	 [0.19683529 0.6857719  0.1173928 ]
2 	 Neutro 	 [0.11851488 0.61981471 0.26167041]
3 	 Neutro 	 [0.23275545 0.74383508 0.02340947]
4 	 Negativo 	 [0.63853877 0.06794565 0.29351558]
5 	 Positivo 	 [0.22854476 0.35588412 0.41557112]


# Random Forest

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

#Criação do modelo
from sklearn.pipeline import make_pipeline
modelo = make_pipeline(TfidfVectorizer(analyzer = "word"),
                       RandomForestClassifier(n_jobs=4, random_state=12))

classifier = GridSearchCV(estimator=modelo, 
                          param_grid={
                              'randomforestclassifier__criterion':['gini', 'entropy'], 
                              'randomforestclassifier__bootstrap':[True,False], 
                              'randomforestclassifier__n_estimators':range(1,11), 
                              'randomforestclassifier__max_depth':range(8,11),
                              'randomforestclassifier__class_weight':['balanced','balanced_subsample'],
                              'tfidfvectorizer__ngram_range':[(1,1), (1,2), (2,2)]
                          },
                          n_jobs=4,
                          cv=10)

X = train_set['Text']
y = train_set['Classificacao']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

classifier.fit(X_train, y_train)

predicoes = classifier.predict(X_test)

np.mean(predicoes == y_test)

0.7395238095238095

In [20]:
from sklearn import metrics
print(metrics.classification_report(y_test, predicoes))
print()
print("Melhores Parâmetros:", classifier.best_params_)

              precision    recall  f1-score   support

    Negativo       0.78      0.63      0.70       411
      Neutro       0.57      0.76      0.65       628
    Positivo       0.88      0.77      0.82      1061

    accuracy                           0.74      2100
   macro avg       0.74      0.72      0.72      2100
weighted avg       0.77      0.74      0.75      2100


Melhores Parâmetros: {'randomforestclassifier__bootstrap': False, 'randomforestclassifier__class_weight': 'balanced', 'randomforestclassifier__criterion': 'entropy', 'randomforestclassifier__max_depth': 9, 'randomforestclassifier__n_estimators': 10, 'tfidfvectorizer__ngram_range': (1, 1)}


In [21]:
#Persistindo o modelo para publicação futura em uma API
from joblib import dump, load
dump(classifier, 'SentimentAnalysisRF.pkl')

['SentimentAnalysisRF.pkl']

In [22]:
#Reabrindo o modelo e validando em algumas frases
modelo = load('SentimentAnalysisRF.pkl')

validacao = ["Existem muitas formas de se aprender, basta querer.",
          "Há dias que o trabalho é muito cansativo.", 
          "Hoje não estou muito bem",
          "Depois do culto você vai ver",
          "Vestidos de heróis dos pés a cabeça para levar um pouco de conhecimento para a humanidade",
          "A Net é muito boa, tem um excelente serviço de telefonia, sucesso de vendas"
         ]
resultado = modelo.predict(validacao)
predicted_proba = modelo.predict_proba(validacao)
for i in range(len(validacao)):
    print(str(i),"\t",resultado[i],"\t",predicted_proba[i])

0 	 Neutro 	 [0.32328157 0.3687935  0.30792493]
1 	 Neutro 	 [0.32328157 0.3687935  0.30792493]
2 	 Negativo 	 [0.37914925 0.33605563 0.28479512]
3 	 Neutro 	 [0.30666577 0.41290992 0.2804243 ]
4 	 Positivo 	 [0.27513599 0.33948954 0.38537447]
5 	 Neutro 	 [0.32328157 0.3687935  0.30792493]


<p> Como podemos ver, ainda há a necessidade de se realizar o Tunning desse modelo, talvez alterando até mesmo a forma de se classificar.</p>
<p> Ao invés de deixá-lo aplicar a classificação automática, utilizar uma função que valide as probabilidades antes de decidir.</p>
<p> Pois como podemos ver no caso das duas primeiras sentenças, o algoritmo ficou com dúvida e aplicou a primeira classe.</p>
<p> Entretanto, para a análise de sentimentos, a dúvida deveria gerar o rótulo Neutro</p>
<p><strong>Dúvidas, entre em contato!</strong></p>